In [4]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import glob
%matplotlib inline

In [5]:
from keras.preprocessing.image import ImageDataGenerator

#Image Data Generator w/ no augmentation
#Scaling for pixels
piece_train_datagen = ImageDataGenerator(
    rescale = 1./255)
piece_test_datagen = ImageDataGenerator(
    rescale = 1./255)
piece_valid_datagen = ImageDataGenerator(
    rescale = 1./255)


#Flow data from directory

piece_train_iter = piece_train_datagen.flow_from_directory(
    directory = '../data/color_data/train',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    seed=42
)

piece_test_iter = piece_test_datagen.flow_from_directory(
    directory = '../data/color_data/test',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    shuffle=False,
    seed=42
)

piece_valid_iter = piece_valid_datagen.flow_from_directory(
    directory = '../data/color_data/valid',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    seed=42
)

Found 1227 images belonging to 3 classes.
Found 335 images belonging to 3 classes.
Found 331 images belonging to 3 classes.


In [6]:
#Define NN architecture

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

piece_model = Sequential()
piece_model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(135, 135, 1)))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(GlobalAveragePooling2D())
piece_model.add(Dense(3, activation='softmax'))


piece_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 135, 135, 16)      80        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 67, 67, 16)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 67, 67, 16)        64        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 67, 67, 32)        2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 33, 33, 32)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 33, 33, 32)        128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 33, 33, 64)        8256      
__________

In [7]:
# compile the model
piece_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

STEP_SIZE_TRAIN = piece_train_iter.n/piece_train_iter.batch_size
STEP_SIZE_VALID = piece_valid_iter.n/piece_valid_iter.batch_size



In [8]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='baseline_model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
piece_hist = piece_model.fit_generator(generator=piece_train_iter, 
                          steps_per_epoch=STEP_SIZE_TRAIN, 
                          validation_data=piece_valid_iter, 
                          validation_steps=STEP_SIZE_VALID,
                          epochs=100, 
                          callbacks=[checkpointer], 
                          verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
 - 12s - loss: 0.5649 - acc: 0.7796 - val_loss: 0.4200 - val_acc: 0.8671

Epoch 00001: val_loss improved from inf to 0.42000, saving model to baseline_model.weights.best.hdf5
Epoch 2/100
 - 11s - loss: 0.3569 - acc: 0.8625 - val_loss: 0.4362 - val_acc: 0.8610

Epoch 00002: val_loss did not improve from 0.42000
Epoch 3/100
 - 13s - loss: 0.2946 - acc: 0.8863 - val_loss: 0.4615 - val_acc: 0.7402

Epoch 00003: val_loss did not improve from 0.42000
Epoch 4/100
 - 13s - loss: 0.2540 - acc: 0.9198 - val_loss: 0.7677 - val_acc: 0.6737

Epoch 00004: val_loss did not improve from 0.42000
Epoch 5/100
 - 15s - loss: 0.2317 - acc: 0.9191 - val_loss: 0.3455 - val_acc: 0.8731

Epoch 00005: val_loss improved from 0.42000 to 0.34551, saving model to baseline_model.weights.best.hdf5
Epoch 6/100
 - 15s - loss: 0.2085 - acc: 0.9335 - val_loss: 0.4635 - val_acc: 0.8761

Epoch 00006: val_loss did not improve from 0.34551
Epoch 7/100
 - 17s - loss:

 - 16s - loss: 0.0042 - acc: 0.9992 - val_loss: 0.0169 - val_acc: 0.9940

Epoch 00054: val_loss did not improve from 0.01271
Epoch 55/100
 - 18s - loss: 0.0075 - acc: 0.9960 - val_loss: 2.6935 - val_acc: 0.7734

Epoch 00055: val_loss did not improve from 0.01271
Epoch 56/100
 - 17s - loss: 0.0121 - acc: 0.9960 - val_loss: 0.0256 - val_acc: 0.9909

Epoch 00056: val_loss did not improve from 0.01271
Epoch 57/100
 - 16s - loss: 0.0200 - acc: 0.9952 - val_loss: 0.0476 - val_acc: 0.9849

Epoch 00057: val_loss did not improve from 0.01271
Epoch 58/100
 - 16s - loss: 0.0032 - acc: 0.9992 - val_loss: 0.0329 - val_acc: 0.9819

Epoch 00058: val_loss did not improve from 0.01271
Epoch 59/100
 - 17s - loss: 0.0368 - acc: 0.9936 - val_loss: 0.0213 - val_acc: 0.9940

Epoch 00059: val_loss did not improve from 0.01271
Epoch 60/100
 - 17s - loss: 0.0207 - acc: 0.9929 - val_loss: 0.5600 - val_acc: 0.8520

Epoch 00060: val_loss did not improve from 0.01271
Epoch 61/100
 - 17s - loss: 0.0045 - acc: 0.998

In [9]:
#Reset test iterator
STEP_SIZE_TEST = piece_test_iter.n/piece_test_iter.batch_size
piece_test_iter.reset()
# load the weights that yielded the best validation accuracy
piece_model.load_weights('baseline_model.weights.best.hdf5')
# evaluate and print test accuracy
score = piece_model.evaluate_generator(generator=piece_test_iter,steps=STEP_SIZE_TEST)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.9970149253731343


In [10]:
piece_test_iter.reset()
piece_pred = piece_model.predict_generator(piece_test_iter,steps=STEP_SIZE_TEST,verbose=1)

11/10 [===============================] - 1s 122ms/step


In [11]:
predicted_class_indices=np.argmax(piece_pred,axis=1)
print(predicted_class_indices)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [12]:
labels = (piece_test_iter.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
truth =  [labels[k] for k in piece_test_iter.classes]

In [13]:
[labels[k] for k in piece_test_iter.classes]


['black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',


In [14]:
piece_test_iter.filenames

['black\\1538774903.7820106.jpg',
 'black\\1538775155.1271548.jpg',
 'black\\1538775387.0960882.jpg',
 'black\\1538776153.1161563.jpg',
 'black\\1538776272.1332486.jpg',
 'black\\1538776570.1745493.jpg',
 'black\\1538776666.2012632.jpg',
 'black\\1538776698.572924.jpg',
 'black\\1538776826.8028915.jpg',
 'black\\1538776828.6866634.jpg',
 'black\\1538776998.5242846.jpg',
 'black\\1538777111.1433806.jpg',
 'black\\1538777117.5942633.jpg',
 'black\\1538777211.243659.jpg',
 'black\\1538777244.9942832.jpg',
 'black\\1538777276.2401419.jpg',
 'black\\1538777285.7202754.jpg',
 'black\\1538777484.4125848.jpg',
 'black\\1538777501.8620265.jpg',
 'black\\1538777647.502205.jpg',
 'black\\1538777804.715925.jpg',
 'black\\1538777891.4931645.jpg',
 'black\\1538777951.5345926.jpg',
 'black\\1538778011.0228102.jpg',
 'black\\1538778091.9695802.jpg',
 'black\\1538778101.779042.jpg',
 'black\\1538778105.383803.jpg',
 'black\\1538778194.7617295.jpg',
 'black\\1538778196.1506379.jpg',
 'black\\1538778215.

In [15]:
filenames=piece_test_iter.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Truth": truth,
                      "Predictions":predictions})

In [16]:
results

,Filename,Truth,Predictions
0,black\1538774903.7820106.jpg,black,black
1,black\1538775155.1271548.jpg,black,black
2,black\1538775387.0960882.jpg,black,black
3,black\1538776153.1161563.jpg,black,black
4,black\1538776272.1332486.jpg,black,black
5,black\1538776570.1745493.jpg,black,black
6,black\1538776666.2012632.jpg,black,black
7,black\1538776698.572924.jpg,black,black
8,black\1538776826.8028915.jpg,black,black
9,black\1538776828.6866634.jpg,black,black


In [14]:
from sklearn.metrics import classification_report
class_report = classification_report(piece_test_iter.classes,predicted_class_indices)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       143
           1       1.00      1.00      1.00        50
           2       1.00      1.00      1.00       142

   micro avg       1.00      1.00      1.00       335
   macro avg       1.00      1.00      1.00       335
weighted avg       1.00      1.00      1.00       335



In [15]:
from sklearn.metrics import confusion_matrix
cmatrix = confusion_matrix(piece_test_iter.classes,predicted_class_indices)
print(cmatrix)
print(labels)

[[143   0   0]
 [  0  50   0]
 [  0   0 142]]
{0: 'black', 1: 'square', 2: 'white'}


In [16]:
from sklearn.metrics import log_loss
log_loss_result = log_loss(piece_test_iter.classes,piece_pred)
print(log_loss_result)

0.013689354841142727


In [18]:
plt.plot(piece_hist.history['acc'])
plt.plot(piece_hist.history['val_acc'])
plt.title('Baseline Color Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

NameError: name 'piece_hist' is not defined

In [19]:
plt.plot(piece_hist.history['loss'])
plt.plot(piece_hist.history['val_loss'])
plt.title('Baseline Color Model Log Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

NameError: name 'piece_hist' is not defined

In [ ]:
import seaborn as sns
ax = sns.heatmap(cmatrix, annot=True, xticklabels=['black','empty','white'],yticklabels=['black','empty','white'])